In [2]:
from openai import OpenAI

def chat_with_gpt(user_prompt, system_prompt, model='gpt-4o', temperature = 1.0):
    
    client = OpenAI()
    
    completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    )

    return completion.choices[0].message.content

In [3]:
txt_list = []
for i in range(9):
    txt_file_path = f"./dummy_dataset/daily_dummy_{i}.txt"
    with open(txt_file_path, 'r', encoding='utf-8') as file:
        txt_data = file.read()
        txt_list.append(txt_data)    

In [58]:
import json

def gen_system_prompt(age, gender, condition, symptoms, experience):
    prompt = f""" 
    You are a person with typical medical condition. Using provided information, perform requested task.

    This is your profile
    Name: Kim
    Age: {age}
    Gender: {gender}
    Medical Condition = {condition}
    Symptoms = {symptoms}
    Past experiences = {experience}

    Whenever you are tasked with performing a task, you must always refer to your profile and execute it from the perspective outlined in the profile
    """

    return prompt

def half_hour_prompt(txt_data):
    prompt = f""" 
    Refer to the given hourly schedule of daily activities to create and organize a 30-minute interval schedule.

    Hourly schedule of daily activity:
    {txt_data}

    Generated answer should be 48 activity divided into every 30 minutes.
    Do not skip the time
    Do not repeat the written activity. Generate new activity that is possible considering two before and after activities 
    Format of the data must follow this: "2025.01.06.TT.MM.SS."  
    Return only shedule
    """

    return prompt

def quarter_hour_prompt(txt_data):
    prompt = f""" 
    Refer to the given 30-minute schedule of daily activities to create and organize a 15-minute interval schedule.

    30-minute schedule of daily activity:
    {txt_data}

    These are the rules you must follow:
    1. Generated answer should be 96 activity divided into every 30 minutes.
    2. Do not skip the time
    3. Do not repeat the written activity. Generate new activity that is possible considering two before and after activities
    4. Format of the data must follow this: "2025.01.06.TT.MM.SS."  
    5. Return only shedule
    """
    
    return prompt

def observation_prompt(txt_data):
    prompt = f"""
    Refer to the given 15-minute schedule of daily activities and create three observations per each schedule.
    Observation should be about what you watch, hear, or speak during each activity

    15-minute schedule of daily activity:
    {txt_data}

    These are the rules you must follow:
    1. Generated observation should written next line of refered activity.
    2. Do not generate obeservation if you were sleeping, but refered activity must be returned
    3. Do not include feelings on your observation
    4. Be specific about your observation
    5. Generated obeservation for each activity should have same time with activity
    6. Format of the data must follow this: "2025.01.06.TT.MM.SS."  
    7. Do not make empty line between activities 
    8. Return only activity and observation
    """
    return prompt

def conversation_prompt(txt_data):
    prompt = f"""
    Refer to the given detailed activity and observation, generate 10 turn conversation if possible
    Conversation should be about current situation

    Detailed activity and observation:
    {txt_data}

    These are the rules you must follow:
    1. You must generate conversation if interaction with someone is occured during refered activity
    2. Do not generate conversation if there was no one to conversate during activity, but refered activity must be returned
    3. Generated conversation should be more than 6 turns but no more than 12 turns
    4. Generated conversation for each activity should have same time with activity
    5. Each speach should be returned with following format:  "2025.01.06.TT.MM.SS. Me: ~~" or "2025.01.06.TT.MM.SS. [name or a word describing the opposite]: ~~"
    7. Never make an empty line between activities, observations, conversations
    8. Return only activity, observation, and conversation
    """

    return prompt

def str_to_json(data, file_path):
    result = []

    lines = data.strip().split('\n')
    for line in lines:
        line = line.strip()
        if line:  
            timestamp, activity = line.split('. ', 1)  
            print(timestamp, activity)
            result.append({"timestamp": timestamp, "activity": activity})

    with open(file_path, 'w', encoding='utf-8') as json_file:
        json.dump(result, json_file, ensure_ascii=False, indent=4)
    

In [60]:
import json
file_path = "./processed_persona_data.json"
with open(file_path, 'r', encoding='utf-8') as file:
    json_data = json.load(file)

count = 0
for patient, txt_data in zip(json_data, txt_list):
    age = patient["Age"]
    gender = patient["Gender"]
    condtion = patient["Existing Medical Conditions"]
    symptoms = patient["Symptoms"]
    experience = patient["Experience"]

    system_prompt = gen_system_prompt(age, gender, condtion, symptoms, experience)
    user_prompt_30min = half_hour_prompt(txt_data)

    half_hour_data = chat_with_gpt(user_prompt_30min, system_prompt, model='gpt-4o')

    user_prompt_15min = quarter_hour_prompt(half_hour_data)
    quarter_hour_data = chat_with_gpt(user_prompt_15min, system_prompt, model='gpt-4o')

    user_prompt_observation = observation_prompt(quarter_hour_data)
    observation_data = chat_with_gpt(user_prompt_observation, system_prompt)

    # user_prompt_conversation = conversation_prompt(observation_data)
    # conversation_data = chat_with_gpt(user_prompt_conversation, system_prompt)

    # print(quarter_hour_data)

    file_path = f'./dummy_dataset/15min_dummy_data_{count}.txt'
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(observation_data)

    count += 1
    print(count)
    break


1


In [61]:
# api아끼려고 conversation 생성 부분 따로 빼냄냄

file_path = f'./dummy_dataset/15min_dummy_data_0.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    txt_data = file.read()

user_prompt_conversation = conversation_prompt(txt_data)
conversation_data = chat_with_gpt(user_prompt_conversation, system_prompt)

file_path = f'./dummy_dataset/conv_data.txt'
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(conversation_data)

In [62]:
# json으로 변환 (생성된 데이터 시작 끝에 """ 가 있으면 수기로 지워야함워야함)

count = 0
for i in range(9):
    file_path = f'./dummy_dataset/15min_dummy_data_{count}.txt'
    file_path = f'./dummy_dataset/conv_data.txt'

    with open(file_path, 'r', encoding='utf-8') as file:
        txt_data = file.read()

    json_path = f'./dummy_dataset/15min_dummy_data_{count}.json'
    str_to_json(txt_data, json_path)
    count += 1 
    break

2025.01.06.03.00.00 I woke up due to back pain
2025.01.06.03.00.00 I noticed the darkness in my room
2025.01.06.03.00.00 I heard a faint ticking of the clock
2025.01.06.03.00.00 I was aware of the quietness around me
2025.01.06.03.15.00 I tried gentle stretching
2025.01.06.03.15.00 I heard a slight crack from my joints
2025.01.06.03.15.00 I watched my shadow on the wall as I stretched
2025.01.06.03.15.00 I heard my muscle softly stretching with each move
2025.01.06.03.30.00 I was trying to relax my back
2025.01.06.03.30.00 I listened to a muffled sound outside
2025.01.06.03.30.00 I felt my muscles gently easing
2025.01.06.03.30.00 I noticed the air feeling cool against my skin
2025.01.06.03.45.00 I took deep breaths and relaxed
2025.01.06.03.45.00 I listened to the rhythmic sound of my breathing
2025.01.06.03.45.00 I felt my chest rising and falling steadily
2025.01.06.03.45.00 I heard the slight rustle of fabric with each breath
2025.01.06.04.00.00 I tried to go back to sleep
2025.01.